## Importing Libraries

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

#### Start Spark SQL

In [2]:
sc = SparkContext(master="local[*]",appName="topmovies")
sql = SQLContext(sc)

#### Loading Data

In [3]:
movies = sql.read.csv("./data/movies.csv",inferSchema=True,header=True)
ratings = sql.read.csv("./data/ratings.csv", inferSchema=True, header=True)

In [4]:
movies.createOrReplaceTempView("movies")

In [5]:
ratings.createOrReplaceTempView("ratings")

In [6]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [7]:
movies.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [9]:
sql.sql("SELECT * FROM movies").show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [10]:
sql.sql("SELECT * FROM ratings").show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
|     1|   1263|   2.0|1260759151|
|     1|   1287|   2.0|1260759187|
|     1|   1293|   2.0|1260759148|
|     1|   1339|   3.5|1260759125|
|     1|   1343|   2.0|1260759131|
|     1|   1371|   2.5|1260759135|
|     1|   1405|   1.0|1260759203|
|     1|   1953|   4.0|1260759191|
|     1|   2105|   4.0|1260759139|
|     1|   2150|   3.0|1260759194|
|     1|   2193|   2.0|1260759198|
|     1|   2294|   2.0|1260759108|
|     1|   2455|   2.5|1260759113|
|     1|   2968|   1.0|1260759200|
|     1|   3671|   3.0|1260759117|
+------+-------+------+----------+
only showing top 20 rows



# Filter Movies That Received at least 10 user reviews

In [18]:
reviewCounts = sql.sql("SELECT movieId, count(*) AS review_counts  FROM ratings GROUP BY movieId")

In [19]:
reviewCounts.show()

+-------+-------------+
|movieId|review_counts|
+-------+-------------+
|   1580|          190|
|   2659|            3|
|   3794|            5|
|   3175|           65|
|    471|           49|
|   1088|           53|
|   1342|           17|
|   1645|           60|
|   2366|           23|
|   6620|           17|
|   8638|           17|
|  96488|            4|
| 160563|            2|
|   7982|            3|
|   1238|           17|
|   1959|           30|
|    463|            7|
|   2122|           11|
|   1591|           15|
|   5518|            1|
+-------+-------------+
only showing top 20 rows



In [21]:
reviewCounts.createOrReplaceTempView("reviewCounts")
reviewCountsGRE10 = sql.sql("SELECT * FROM reviewCounts WHERE review_counts >= 10")

In [22]:
reviewCountsGRE10.show()

+-------+-------------+
|movieId|review_counts|
+-------+-------------+
|   1580|          190|
|   3175|           65|
|    471|           49|
|   1088|           53|
|   1342|           17|
|   1645|           60|
|   2366|           23|
|   6620|           17|
|   8638|           17|
|   1238|           17|
|   1959|           30|
|   2122|           11|
|   1591|           15|
|  44022|           20|
|   2142|           12|
|   2866|           10|
|  68135|           11|
|   3997|           10|
|   1721|          164|
|    858|          200|
+-------+-------------+
only showing top 20 rows



In [23]:
reviewCountsGRE10.count()

2245

In [24]:
reviewCountsGRE10.createOrReplaceTempView("reviewCountsGRE10")

### Join reviewCountsGRE10 with ratings data

In [27]:
df = sql.sql("SELECT r1.movieId, r1.review_counts, r2.userId, r2.rating FROM reviewCountsGRE10 r1 JOIN ratings r2 USING(movieId)")

In [28]:
df.show()

+-------+-------------+------+------+
|movieId|review_counts|userId|rating|
+-------+-------------+------+------+
|     31|           42|     1|   2.5|
|   1029|           42|     1|   3.0|
|   1061|           33|     1|   3.0|
|   1129|           48|     1|   2.0|
|   1172|           46|     1|   4.0|
|   1263|           48|     1|   2.0|
|   1287|           46|     1|   2.0|
|   1293|           46|     1|   2.0|
|   1339|           52|     1|   3.5|
|   1343|           39|     1|   2.0|
|   1371|           47|     1|   2.5|
|   1405|           46|     1|   1.0|
|   1953|           46|     1|   4.0|
|   2105|           47|     1|   4.0|
|   2150|           36|     1|   3.0|
|   2193|           42|     1|   2.0|
|   2294|           53|     1|   2.0|
|   2455|           47|     1|   2.5|
|   2968|           43|     1|   1.0|
|   3671|           62|     1|   3.0|
+-------+-------------+------+------+
only showing top 20 rows



In [29]:
df.count()

81915

In [30]:
df.createOrReplaceTempView("df")

## TOP 20 movies with highest average ratings

In [38]:
avgRatings = sql.sql("SELECT movieId, AVG(rating) AS AvgRating ,AVG(review_counts) AS review_counts FROM df GROUP BY movieId ORDER BY AvgRating DESC LIMIT 20")

In [39]:
avgRatings.show()

+-------+-----------------+-------------+
|movieId|        AvgRating|review_counts|
+-------+-----------------+-------------+
|   1939|4.636363636363637|         11.0|
|   3469|4.541666666666667|         12.0|
|    858|           4.4875|        200.0|
|    318|4.487138263665595|        311.0|
|   1948|4.458333333333333|         12.0|
|   8132|4.454545454545454|         11.0|
|   1945|4.448275862068965|         29.0|
|   1147|           4.4375|         16.0|
|    926|4.434210526315789|         38.0|
|   1217|4.423076923076923|         26.0|
|    969|             4.42|         50.0|
|   3035|4.411764705882353|         17.0|
|   1066|4.409090909090909|         11.0|
|   2203|              4.4|         10.0|
|   2064|4.392857142857143|         42.0|
|    913|4.387096774193548|         62.0|
|   7502|4.386363636363637|         22.0|
|   1221|4.385185185185185|        135.0|
|    905|             4.38|         25.0|
|     50|4.370646766169155|        201.0|
+-------+-----------------+-------

In [40]:
avgRatings.createOrReplaceTempView("avgRatings")

## Finding top20 movies

In [52]:
result = sql.sql("SELECT a.movieId, m.title AS `Movie Name` , CAST(a.AvgRating AS DECIMAL(5, 2) )AS `Average Rating`,a.review_counts AS `Total number of ratings` FROM avgRatings a JOIN movies m USING(movieId) ORDER BY `Average Rating`, `Total number of ratings` DESC")

In [53]:
result.show(truncate=False)

+-------+-----------------------------------+--------------+-----------------------+
|movieId|Movie Name                         |Average Rating|Total number of ratings|
+-------+-----------------------------------+--------------+-----------------------+
|50     |Usual Suspects, The (1995)         |4.37          |201.0                  |
|905    |It Happened One Night (1934)       |4.38          |25.0                   |
|1221   |Godfather: Part II, The (1974)     |4.39          |135.0                  |
|913    |Maltese Falcon, The (1941)         |4.39          |62.0                   |
|2064   |Roger & Me (1989)                  |4.39          |42.0                   |
|7502   |Band of Brothers (2001)            |4.39          |22.0                   |
|2203   |Shadow of a Doubt (1943)           |4.40          |10.0                   |
|3035   |Mister Roberts (1955)              |4.41          |17.0                   |
|1066   |Shall We Dance (1937)              |4.41          |11.0 

In [54]:
sc.stop()